<a href="https://colab.research.google.com/github/YASHASM1502/carbon-footprint-optimization-in-supply-chain-logitics-/blob/main/carbon_footprint_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [8]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import numpy as np

# Example data: distances between locations (in km)
distance_matrix = [
    [0, 20, 18, 30, 10],
    [20, 0, 15, 35, 25],
    [18, 15, 0, 28, 22],
    [30, 35, 28, 0, 12],
    [10, 25, 22, 12, 0]
]

# CO₂ emission rate per km (in kg CO2/km)
emission_rate = 0.21  # for a diesel van

# Define number of vehicles and depot
num_vehicles = 1
depot = 0

def create_data_model():
    """Stores the data for the problem."""
    data = {
        'distance_matrix': distance_matrix,
        'num_vehicles': num_vehicles,
        'depot': depot
    }
    return data

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    total_distance = 0
    total_emissions = 0
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    while not routing.IsEnd(index):
        plan_output += f'{manager.IndexToNode(index)} -> '
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        distance = routing.GetArcCostForVehicle(previous_index, index, 0)
        total_distance += distance
        total_emissions += distance * emission_rate
    plan_output += f'{manager.IndexToNode(index)}\n'
    plan_output += f'Total distance: {total_distance} km\n'
    plan_output += f'Total CO₂ emissions: {total_emissions:.2f} kg\n'
    print(plan_output)

def main():
    data = create_data_model()

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['distance_matrix'][from_node][to_node])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Set search parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        print_solution(manager, routing, solution)
    else:
        print("No solution found.")

if __name__ == '__main__':
    main()


Route for vehicle 0:
0 -> 4 -> 3 -> 2 -> 1 -> 0
Total distance: 85 km
Total CO₂ emissions: 17.85 kg

